In [1]:
import os
import pandas as pd

In [17]:
outfile = "LN-shared-BvS.xlsx"

## Read data

In [2]:
myfiles = [x for x in os.listdir(".") if x.startswith("LN-aa-run")]
myfiles

['LN-aa-run81-2019-10-21-clones-script20150427.csv',
 'LN-aa-run169-2019-09-10-clones-script20150427.csv',
 'LN-aa-run160-2019-09-10-clones-script20150427.csv',
 'LN-aa-run162-2019-10-29-clones-script20150427.csv',
 'LN-aa-run74-2019-10-21-clones-script20150427.csv',
 'LN-aa-run98-2019-10-17-clones-script20150427.csv',
 'LN-aa-run142-2019-10-17-clones-script20150427.csv',
 'LN-aa-run181-2019-09-10-clones-script20150427.csv',
 'LN-aa-run141-2019-09-10-clones-script20150427.csv']

In [3]:
cols = ['ID', 'CDR3', 'count']
df = pd.read_csv(myfiles[0])
df = df[cols]
df.head()

,ID,CDR3,count
0,LN92,CARDAYSSGWYYFDYWGQGTLVT,2589
1,LN92,CARGGVYCSGGSCFYGMDVWGQGTTVT,866
2,LN92,CAKDRSSWGDQNSFDYWGQGTLVT,529
3,LN92,CARDWGYSGYDPGGRGFDYWGQGTLVT,328
4,LN92,CARDAYSSGWYYFDYWGQGTLVT,246


In [4]:
for myfile in myfiles[1:]:
    df_tmp = pd.read_csv(myfile)
    df_tmp = df_tmp[cols]
    df = df.append(df_tmp)
df.tail()

,ID,CDR3,count
66044,LN78,CAGMNYDFWGGWGYGMDVWGQGTTVT,1
66045,LN78,CAGMNYDFWSGWGYGMDVWGQGTTVT,1
66046,LN78,CARHFAGLGSSYHSLGMDVWGLGTTVT,1
66047,LN78,CARGNGIFGLVIIGLWGQGTLVT,1
66048,LN78,CVVRMPSDFWGQGTLVT,1


In [5]:
rosetta = pd.read_csv("rosetta-anonymized-complete.csv")
rosetta.head()

,Group,Pt-nr,Tissue,Sample,ID,Visit,Patient
0,HC,1,LN,BB4726,LN87,first,HC-1
1,HC,2,LN,BB4727,LN88,first,HC-2
2,HC,3,LN,BB5491,LN14,first,HC-3
3,HC,4,LN,BB5492,LN13,first,HC-4
4,HC,5,LN,BB5531,LN25,first,HC-5


In [6]:
rosetta = rosetta[(rosetta['Tissue'] == "PB") & (rosetta['Visit'] == "first")]
rosetta

,Group,Pt-nr,Tissue,Sample,ID,Visit,Patient
10,HC,1,PB,BB4726,LN89,first,HC-1
11,HC,2,PB,BB4727,LN90,first,HC-2
12,HC,3,PB,BB5491,LN50,first,HC-3
13,HC,4,PB,BB5492,LN51,first,HC-4
14,HC,5,PB,BB5531,LN52,first,HC-5
15,HC,6,PB,BB6022,LN53,first,HC-6
16,HC,7,PB,BB6102,LN54,first,HC-7
17,HC,8,PB,BB6282,LN55,first,HC-8
18,HC,9,PB,BB6283,LN56,first,HC-9
19,HC,10,PB,BB6430,LN57,first,HC-10


In [7]:
df = pd.merge(df, rosetta, on="ID")
df.head()

,ID,CDR3,count,Group,Pt-nr,Tissue,Sample,Visit,Patient
0,LN95,CARVITMVQGVKLYYMDVWGKGTTVT,7886,RA-plus,4,PB,BB3208,first,RA-plus-4
1,LN95,CARVITMVQGVKLYYMDVWGKGTTVT,698,RA-plus,4,PB,BB3208,first,RA-plus-4
2,LN95,CARVAHYDFWSGFRWFDPWGQGTLVT,594,RA-plus,4,PB,BB3208,first,RA-plus-4
3,LN95,CARDRDGYNYDYWGQGTLVT,446,RA-plus,4,PB,BB3208,first,RA-plus-4
4,LN95,CARVITMVQGVKLYYMDVWGKGTTVT,383,RA-plus,4,PB,BB3208,first,RA-plus-4


In [8]:
df = df.groupby(["Patient", "ID", "CDR3"]).agg({'count': sum})
df = df.reset_index()
df.head()

,Patient,ID,CDR3,count
0,HC-1,LN89,CAAAPDFWSGYYYYYGMDVWGQGTTVT,1
1,HC-1,LN89,CAADRGNSNEGVGVYYGMDVWGQGTTVT,1
2,HC-1,LN89,CAADSDIVVVPAANYYYYYGMDVWGQGTTVT,2
3,HC-1,LN89,CAADYYDSSGYPSYDYWGQGTLVT,1
4,HC-1,LN89,CAAEVGDGSRVYWGQGTLVT,1


In [9]:
df_total = df.groupby("ID").agg({"count": sum})
df_total = df_total.reset_index()
df_total = df_total.rename(columns={'count': 'totalreads'})
df_total.head()

,ID,totalreads
0,LN29,9678
1,LN30,9761
2,LN31,9739
3,LN32,10070
4,LN33,9807


In [10]:
df = pd.merge(df, df_total, on="ID")
df.head()

,Patient,ID,CDR3,count,totalreads
0,HC-1,LN89,CAAAPDFWSGYYYYYGMDVWGQGTTVT,1,11451
1,HC-1,LN89,CAADRGNSNEGVGVYYGMDVWGQGTTVT,1,11451
2,HC-1,LN89,CAADSDIVVVPAANYYYYYGMDVWGQGTTVT,2,11451
3,HC-1,LN89,CAADYYDSSGYPSYDYWGQGTLVT,1,11451
4,HC-1,LN89,CAAEVGDGSRVYWGQGTLVT,1,11451


In [11]:
df.tail()

,Patient,ID,CDR3,count,totalreads
186030,RA-plus-9,LN31,YAREATGWASRVFDYWGQGTLVT,1,9739
186031,RA-plus-9,LN31,YARGGHGDAVVHPAYNWGQGTLVT,1,9739
186032,RA-plus-9,LN31,YATSYYDSSGDLTWYYFDNWGQGTLVT,1,9739
186033,RA-plus-9,LN31,YAVLADGMDVWGQGTTVT,1,9739
186034,RA-plus-9,LN31,YTRKLRFLEWLPIDFDYWGQGTLVT,1,9739


In [12]:
df['perc'] = 100 * df['count'] / df['totalreads']
df.head()

,Patient,ID,CDR3,count,totalreads,perc
0,HC-1,LN89,CAAAPDFWSGYYYYYGMDVWGQGTTVT,1,11451,0.008733
1,HC-1,LN89,CAADRGNSNEGVGVYYGMDVWGQGTTVT,1,11451,0.008733
2,HC-1,LN89,CAADSDIVVVPAANYYYYYGMDVWGQGTTVT,2,11451,0.017466
3,HC-1,LN89,CAADYYDSSGYPSYDYWGQGTLVT,1,11451,0.008733
4,HC-1,LN89,CAAEVGDGSRVYWGQGTLVT,1,11451,0.008733


In [13]:
ids = list(df['ID'].unique())
ids[:10]

['LN89',
 'LN57',
 'LN90',
 'LN50',
 'LN51',
 'LN52',
 'LN53',
 'LN54',
 'LN55',
 'LN56']

In [14]:
def directionShared(df, idA, idB):
    df_A = df[df['ID'] == idA]
    df_B = df[df['ID'] == idB]
    df_pair = pd.merge(df_A, df_B, on="CDR3")
    sum_A = df_pair["perc_x"].sum()
    sum_B = df_pair["perc_y"].sum()
    if sum_B > sum_A:
        idA, idB = idB, idA
        sum_A, sum_B = sum_B, sum_A
    return(idA, idB, sum_A, sum_B)
directionShared(df, "LN98", "LN99")

('LN99', 'LN98', 44.3471188137396, 27.968337730870715)

In [15]:
sources = list()
targets = list()
impacts_source = list()
impacts_target = list()
for i in range(len(ids)-1):
    for j in range(i+1, len(ids)):
        idA = ids[i]
        idB = ids[j]
        (source, target, impact_source, impact_target) = directionShared(df, idA, idB)
        if impact_source == 0 and impact_target == 0:
            continue   # skip if there is no overlap
        sources.append(source)
        targets.append(target)
        impacts_source.append(impact_source)
        impacts_target.append(impact_target)
df_shared = pd.DataFrame({'source': sources, 'target': targets, 'impact_source': impacts_source, 'impact_target': impacts_target})
df_shared.head()

,source,target,impact_source,impact_target
0,LN57,LN89,0.087117,0.069863
1,LN90,LN89,0.208929,0.130993
2,LN53,LN89,0.010669,0.008733
3,LN54,LN89,0.024219,0.017466
4,LN55,LN89,0.022274,0.008733


In [16]:
# sort on impact on the target and then on impact of the source
df_shared = df_shared.sort_values(by=["impact_target", "impact_source"], ascending=False)
df_shared

,source,target,impact_source,impact_target
175,LN97,LN98,36.677503,33.056917
176,LN99,LN97,44.872677,32.726152
215,LN99,LN98,44.347119,27.968338
53,LN99,LN90,12.844898,1.011656
50,LN97,LN90,7.067593,0.923686
52,LN98,LN90,5.773338,0.879701
184,LN97,LN95,10.150207,0.464379
224,LN98,LN95,7.884156,0.453052
236,LN99,LN95,17.956579,0.396421
1,LN90,LN89,0.208929,0.130993


In [18]:
df_shared.to_excel(outfile)